In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from datetime import datetime

DATA_PATH = "Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
TARGET_STATE = "GA"

In [2]:
def get_zhvi_columns(df):
    return df.columns[df.columns.str.match(r'^\d{4}-\d{2}-\d{2}$')]

def compute_yoy(series, year):
    """Compute YoY growth from Dec of previous year to Dec of current year."""
    try:
        prev = series.loc[f"{year-1}-12-31"]
        curr = series.loc[f"{year}-12-31"]
        if pd.notna(prev) and prev != 0:
            return 100 * (curr - prev) / prev
    except:
        return np.nan

def cagr(series):
    if len(series) < 2 or series.iloc[0] <= 0:
        return np.nan
    years = len(series) / 12
    return ((series.iloc[-1] / series.iloc[0]) ** (1 / years) - 1) * 100

def avg_monthly_growth(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.mean()

def volatility(series):
    monthly_pct = series.pct_change()
    return 100 * monthly_pct.std()

def longest_nan_streak(series):
    is_null = series.isnull()
    max_streak = current = 0
    for val in is_null:
        if val:
            current += 1
            max_streak = max(max_streak, current)
        else:
            current = 0
    return max_streak


In [3]:
# load data and clean data

df = pd.read_csv(DATA_PATH)
df = df[df["State"] == TARGET_STATE].copy()
zhvi_cols = get_zhvi_columns(df)
print(f"loaded {len(df)} ZIPs from {TARGET_STATE}")

# fill missing Metro using City
ambiguous_cities = {"Boston", "Tifton"} # these cities had multiple metros so i choose to exclude them to avoid ambiguity
safe_city_to_metro = (
    df[df["City"].notna() & df["Metro"].notna() & ~df["City"].isin(ambiguous_cities)]
    .groupby("City")["Metro"].agg(lambda x: x.mode().iloc[0]).to_dict()
)
df["Metro"] = df.apply(
    lambda row: safe_city_to_metro.get(row["City"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)
county_to_metro = (
    df[df["Metro"].notna()][["CountyName", "Metro"]]
    .drop_duplicates()
    .groupby("CountyName")["Metro"].agg(lambda x: x.mode().iloc[0])
    .to_dict()
)
df["Metro"] = df.apply(
    lambda row: county_to_metro.get(row["CountyName"], row["Metro"]) if pd.isna(row["Metro"]) else row["Metro"],
    axis=1
)

# drop rows still missing Metro or City
df = df[df["Metro"].notna() & df["City"].notna()].copy()
print(f"remaining after drop: {len(df)} rows")

df.rename(columns={"RegionName": "ZIP"}, inplace=True)
df["ZIP"] = df["ZIP"].astype(str)


loaded 665 ZIPs from GA
remaining after drop: 558 rows


In [4]:
# build rolling rows with per-year features and YoY target

long_rows = []
zhvi_cols = get_zhvi_columns(df)

for _, row in df.iterrows():
    series = row[zhvi_cols].astype(float)
    series.index = pd.to_datetime(series.index)

    if longest_nan_streak(series) > 12:
        continue

    series = series.interpolate(method="linear", limit_direction="both")

    for year in range(2011, 2024):  # only keep years where ACS is available
        try:
            cutoff_date = f"{year}-12-31"
            future_date = f"{year + 1}-12-31"

            if cutoff_date not in series.index or future_date not in series.index:
                continue
            if f"{year - 1}-12-31" not in series.index:
                continue  

            subset = series[series.index <= cutoff_date]
            if len(subset) < 24:
                continue

            final_value = subset.iloc[-1]
            avg_yoy = subset.pct_change(periods=12).mean() * 100
            med_yoy = subset.pct_change(periods=12).median() * 100
            vol = volatility(subset)
            cagr_val = cagr(subset)
            neg_years = sum((subset.pct_change(12) < 0).fillna(False))

            yoy_target = compute_yoy(series, year + 1)  # future year
            yoy_last_year = compute_yoy(series, year)   # current year

            if pd.isna(yoy_target) or pd.isna(yoy_last_year):
                continue

            clean_county = str(row["CountyName"]).replace(" County", "").strip()

            long_rows.append({
                "ZIP": row["ZIP"],
                "Year": year,
                "Metro": row["Metro"],
                "CountyName": clean_county,
                "StateSizeRank": row["SizeRank"],
                "FinalZHVI": final_value,
                "CAGR": cagr_val,
                "AvgMonthlyGrowth": avg_monthly_growth(subset),
                "AvgYoYGrowth": avg_yoy,
                "MedianYoYGrowth": med_yoy,
                "Volatility": vol,
                "NegativeGrowthYears": neg_years,
                "YoY_LastYear": yoy_last_year,
                "YoY_target": yoy_target
            })

        except Exception:
            continue

rolling_df = pd.DataFrame(long_rows)
print(f"created long-format dataset: {rolling_df.shape[0]} rows")
display(rolling_df.head())


created long-format dataset: 5343 rows


,ZIP,Year,Metro,CountyName,StateSizeRank,FinalZHVI,CAGR,AvgMonthlyGrowth,AvgYoYGrowth,MedianYoYGrowth,Volatility,NegativeGrowthYears,YoY_LastYear,YoY_target
0,30044,2011,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,96925.609814,-2.728627,-0.228476,-2.240039,2.075935,0.825100,48,-16.386504,1.357623
1,30044,2012,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,98241.493964,-2.420235,-0.201841,-2.725794,1.916042,0.827611,59,1.357623,28.758114
2,30044,2013,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,126493.894446,-0.468436,-0.034224,-1.106771,2.075935,1.015673,59,28.758114,12.641864
3,30044,2014,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,142485.080180,0.356019,0.034933,0.484466,2.118797,1.017112,59,12.641864,9.107272
4,30044,2015,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,155461.584010,0.881799,0.078549,1.052938,2.188492,0.999959,59,9.107272,9.632784


In [5]:
# merge in ACS 1-Year Estimates per row's year

import requests

API_KEY = "b06c64cd8eadc4fa292ce05e788144d98223fa3f"
acs_vars = [
    "B25003_001E", "B25003_003E", "B08201_001E", "B08201_002E",
    "B15003_001E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E",
    "B23025_003E", "B23025_005E", "B17001_001E", "B17001_002E", "B01003_001E",
    "B01001_003E", "B01001_027E", "B01001_004E", "B01001_028E", "B01001_005E", "B01001_029E",
    "B01001_006E", "B01001_030E", "B01001_007E", "B01001_031E", "B01001_008E", "B01001_032E",
    "B01001_009E", "B01001_033E", "B01001_010E", "B01001_034E", "B01001_020E", "B01001_044E",
    "B11001_001E", "B11001_008E", "B11016_001E", "B11016_010E"
]

valid_years = [y for y in sorted(rolling_df["Year"].unique()) if y >= 2001 and y != 2020]
all_yearly_acs = []

for year in valid_years:
    print(f"fetching ACS for year {year}")
    try:
        url = f"https://api.census.gov/data/{year}/acs/acs1?get=NAME,{','.join(acs_vars)}&for=county:*&in=state:13&key={API_KEY}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed for {year}: {response.text}")
            continue

        acs_data = response.json()
        columns = acs_data[0]
        rows = acs_data[1:]
        acs_df = pd.DataFrame(rows, columns=columns)

        acs_df["state"] = acs_df["state"].astype(str)
        acs_df["county"] = acs_df["county"].astype(str)
        acs_df["GEOID"] = acs_df["state"].str.zfill(2) + acs_df["county"].str.zfill(3)

        # normalize CountyName
        acs_df["CountyName"] = acs_df["NAME"].str.replace(" County, Georgia", "", regex=False).str.strip()

        for col in acs_vars:
            acs_df[col] = pd.to_numeric(acs_df[col], errors="coerce")

        acs_df["Pct_Renter_Occupied"] = 100 * acs_df["B25003_003E"] / acs_df["B25003_001E"]
        acs_df["Pct_Bachelors_Or_Higher"] = 100 * (
            acs_df["B15003_022E"] + acs_df["B15003_023E"] + acs_df["B15003_024E"] + acs_df["B15003_025E"]
        ) / acs_df["B15003_001E"]
        acs_df["Unemployment_Rate"] = 100 * acs_df["B23025_005E"] / acs_df["B23025_003E"]
        acs_df["Pct_Below_Poverty"] = 100 * acs_df["B17001_002E"] / acs_df["B17001_001E"]
        acs_df["Pct_No_Vehicle"] = 100 * acs_df["B08201_002E"] / acs_df["B08201_001E"]
        acs_df["Pct_One_Person_HH"] = 100 * acs_df["B11001_008E"] / acs_df["B11001_001E"]
        acs_df["Pct_4plus_HH"] = 100 * acs_df["B11016_010E"] / acs_df["B11016_001E"]
        acs_df["Pct_Age_0_17"] = 100 * (
            acs_df["B01001_003E"] + acs_df["B01001_027E"] + acs_df["B01001_004E"] + acs_df["B01001_028E"] +
            acs_df["B01001_005E"] + acs_df["B01001_029E"] + acs_df["B01001_006E"] + acs_df["B01001_030E"] +
            acs_df["B01001_007E"] + acs_df["B01001_031E"]
        ) / acs_df["B01003_001E"]
        acs_df["Pct_Age_18_34"] = 100 * (
            acs_df["B01001_008E"] + acs_df["B01001_032E"] + acs_df["B01001_009E"] + acs_df["B01001_033E"] +
            acs_df["B01001_010E"] + acs_df["B01001_034E"]
        ) / acs_df["B01003_001E"]
        acs_df["Pct_Age_65plus"] = 100 * (
            acs_df["B01001_020E"] + acs_df["B01001_044E"]
        ) / acs_df["B01003_001E"]

        acs_df["Year"] = year
        subset = acs_df[[
            "CountyName", "Year", "Pct_Renter_Occupied", "Pct_Bachelors_Or_Higher", "Unemployment_Rate",
            "Pct_Below_Poverty", "Pct_No_Vehicle", "Pct_One_Person_HH", "Pct_4plus_HH",
            "Pct_Age_0_17", "Pct_Age_18_34", "Pct_Age_65plus"
        ]].copy()

        all_yearly_acs.append(subset)

    except Exception as e:
        print(f"Error processing year {year}: {e}")

acs_all = pd.concat(all_yearly_acs, ignore_index=True)

# merge 
rolling_df["CountyName"] = rolling_df["CountyName"].str.strip().str.title()
acs_all["CountyName"] = acs_all["CountyName"].str.strip().str.title()
rolling_df = rolling_df[rolling_df["Year"] != 2020].copy()
rolling_df = rolling_df.merge(acs_all, on=["CountyName", "Year"], how="left", indicator=True)
rolling_df.drop(columns=["_merge"], inplace=True)

print(f"final merged dataset with acs data: {rolling_df.shape}")
display(rolling_df.head(3))



fetching ACS for year 2011
fetching ACS for year 2012
fetching ACS for year 2013
fetching ACS for year 2014
fetching ACS for year 2015
fetching ACS for year 2016
fetching ACS for year 2017
fetching ACS for year 2018
fetching ACS for year 2019
fetching ACS for year 2021
fetching ACS for year 2022
fetching ACS for year 2023
final merged dataset with acs data: (4932, 24)


,ZIP,Year,Metro,CountyName,StateSizeRank,FinalZHVI,CAGR,AvgMonthlyGrowth,AvgYoYGrowth,MedianYoYGrowth,...,Pct_Renter_Occupied,Pct_Bachelors_Or_Higher,Unemployment_Rate,Pct_Below_Poverty,Pct_No_Vehicle,Pct_One_Person_HH,Pct_4plus_HH,Pct_Age_0_17,Pct_Age_18_34,Pct_Age_65plus
0,30044,2011,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,96925.609814,-2.728627,-0.228476,-2.240039,2.075935,...,33.588916,33.965298,11.391468,15.718932,2.986379,19.359051,19.359051,31.335089,6.017885,1.235967
1,30044,2012,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,98241.493964,-2.420235,-0.201841,-2.725794,1.916042,...,34.406210,33.555604,9.447880,14.841315,2.582147,19.612641,19.612641,30.905437,6.243483,1.234612
2,30044,2013,"Atlanta-Sandy Springs-Alpharetta, GA",Gwinnett,21,126493.894446,-0.468436,-0.034224,-1.106771,2.075935,...,32.960599,33.020367,9.552809,13.557450,3.395815,19.240722,19.240722,30.790849,6.430320,1.335732


In [6]:
# export dataset with one-hot encoding

# remove rows with missing target
rolling_df = rolling_df[rolling_df["YoY_target"].notna()].copy()

# group rare Metro and CountyName
def group_rare(series, min_count=15):
    counts = series.value_counts()
    return series.apply(lambda x: x if counts[x] >= min_count else "Other")

rolling_df["Metro"] = group_rare(rolling_df["Metro"])
rolling_df["CountyName"] = group_rare(rolling_df["CountyName"])

# one-hot encode categorical columns
rolling_df = pd.get_dummies(rolling_df, columns=["Metro", "CountyName"], drop_first=True)

# drop NA and reset index
rolling_df = rolling_df.dropna().reset_index(drop=True)

# export to CSV
rolling_df.to_csv("final_processed_dataset.csv", index=False)
print(f"exported dataset with {rolling_df.shape[0]} rows and {rolling_df.shape[1]} columns.")


exported dataset with 3009 rows and 139 columns.
